# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import face_recognition
from PIL import Image
import keras
from mtcnn.mtcnn import MTCNN
import dlib

# Importing models

In [ ]:
names = ['shivani_di', 'harsh', 'chinki_di', 'manu_bhaiya', 'ashu_bhaiya', 'shruti_di', 'tanu', 'prerna', 'puneeth', 'priya', 'siddhant', 'chitiz', 'papa', 'neha_di', 'rachi', 'nikki_di', 'mumma', 'celina', 'shubhangi', 'Vishal', 'akash_bhai', 'annu_di', 'happy_bhaiya', 'sunny_bhaiya']

model1 = keras.models.load_model('best_fit3')

base_dir = os.getcwd()
prototxt_path = os.path.join(base_dir + '/deploy.prototxt')
caffemodel_path = os.path.join(base_dir + '/weights.caffemodel')

# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)
base_dir = os.getcwd()
detector = MTCNN()
cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height
a = 0

# Video capture

In [ ]:
while(True):
	ret, frame = cap.read()
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
	model.setInput(blob)
	detections = model.forward()

	# Identify each face
	for i in range(0, detections.shape[2]):
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		confidence = detections[0, 0, i, 2]
		# If confidence > 0.5, save it as a separate file
		if (confidence > 0.5):
			try:
				cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 255, 255), 1)
				image = frame[startY:endY, startX:endX]
				data = cv2.resize(image, (150, 150))
				arrx = np.array(face_recognition.face_encodings(data, known_face_locations=[(0, 150, 150, 0)])[0]).reshape((1,128))
				#im_pillow = np.array(img).reshape((1,150,150,3))
				predictions = model1.predict(arrx,batch_size=128)
				classes = np.argmax(predictions, axis = 1)
				cv2.putText(frame, names[classes[0]], (startX, startY), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
				#print (names[classes[0]])
			except:
				pass
		#output = cv2.resize(frame, (960, 540))
		cv2.imshow('frame', frame)
	k = cv2.waitKey(30) & 0xff
	if k == 27: # press 'ESC' to quit
		break
cap.release()
cv2.destroyAllWindows()